<a href="https://colab.research.google.com/github/mnocerino23/Wildfire-Forecaster/blob/main/NOAA_weatherdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I requested monthly weather data from the state of California's various weather stations from 2001-2019. In this notebook, I engineer additional weather features, map each station to its coordinates, then find the closest station to each fire that occured based on the coordinates

In [12]:
#Read in the csv file from my google drive
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
weather = pd.read_csv('/content/drive/MyDrive/Data_Science_Projects/Wildfires/NOAA_California.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Weather Features:


1.   Station - Weather Station in California
2.   Date - Year and month of the report
3.   AWND - Average Monthly Wind Speed
4.   CDSD - Cooling Degree Days Season to Date
5.   CLDD - Cooling Degree Days
6.   DP10 - Days with over 0.10 inches of percipitation
7.   DX90 - Days with temperature over 90
8.   DYTS - Number of days with thunderstorms
9.   EMXP - Extreme max percipitation
10.  EMXT - Extreme max temperature
11.  PRCP - Monthly Percipitation
12.  SNOW - Total Monthly Snowfall
13.  TAVG - Average Temperature
14.  TMAX - Max Temperature
15.  TMIN - Min Temperature






In [13]:
weather.shape

(19239, 15)

In [14]:
weather.head(8)

,STATION,DATE,AWND,CDSD,CLDD,DP10,DX90,DYTS,EMXP,EMXT,PRCP,SNOW,TAVG,TMAX,TMIN
0,USW00023129,2000-08,5.6,953.0,323.0,0.0,6.0,NaN,0.00,97.0,0.00,NaN,75.4,84.3,66.5
1,USW00023129,2000-09,5.1,1175.0,221.0,0.0,5.0,NaN,0.00,96.0,0.00,NaN,72.4,82.5,62.3
2,USW00023129,2000-10,4.7,1208.0,33.0,4.0,0.0,1.0,1.81,79.0,2.30,NaN,64.7,71.9,57.5
3,USW00023129,2000-11,3.6,1208.0,0.0,0.0,0.0,1.0,0.00,80.0,0.00,NaN,56.9,68.1,45.8
4,USW00023129,2000-12,3.1,1209.0,0.0,0.0,0.0,NaN,0.00,80.0,0.00,NaN,57.1,68.3,46.0
5,USW00023129,2001-01,3.6,0.0,0.0,5.0,0.0,1.0,0.71,82.0,2.11,NaN,52.4,63.1,41.7
6,USW00023129,2001-02,5.1,1.0,1.0,8.0,0.0,2.0,1.93,87.0,5.79,NaN,52.5,61.5,43.4
7,USW00023129,2001-03,5.1,7.0,6.0,1.0,0.0,1.0,0.25,85.0,0.26,NaN,59.0,66.3,51.8


Read in a CSV file that has the California NOAA weather stations and their coordinates and split it into latitude and longitude

In [15]:
stations = pd.read_csv('/content/drive/MyDrive/Data_Science_Projects/Wildfires/NOAA_Stations.csv')

In [16]:
stations.rename({'Unnamed: 3': 'Longitude', 'Coordinates':'Latitude'}, axis = 1, inplace = True)
for index, row in stations.iterrows():
  s = stations.at[index,'Latitude'].split(' ')
  stations.at[index,'Longitude'] = s[1]
  stations.at[index,'Latitude'] = s[0]
stations.head(10)

,Unnamed: 0,Station,Latitude,Longitude
0,0,USW00023129,33.8117,-118.1464
1,1,USW00093111,34.1167,-119.1167
2,2,USW00093112,32.7000,-117.2000
3,3,USW00093115,32.5667,-117.1167
4,4,USW00093116,33.2500,-119.4500
5,5,USW00003167,33.9228,-118.3342
6,6,USW00023293,37.3592,-121.9239
7,7,USW00023174,33.9381,-118.3889
8,8,USW00023130,34.2097,-118.4892
9,9,USW00093241,38.3775,-121.9575


In [17]:
#Dictionaries mapping the station to its latitude and longitude
lat = dict(zip(stations['Station'],stations['Latitude']))
long = dict(zip(stations['Station'],stations['Longitude']))

In [18]:
weather['Latitude'] = ''
weather['Longitude'] = ''
for index, row in weather.iterrows():
  weather.at[index, 'Latitude'] = lat[weather.at[index,'STATION']] 
  weather.at[index, 'Longitude'] = long[weather.at[index,'STATION']] 

In [19]:
weather.head()

,STATION,DATE,AWND,CDSD,CLDD,DP10,DX90,DYTS,EMXP,EMXT,PRCP,SNOW,TAVG,TMAX,TMIN,Latitude,Longitude
0,USW00023129,2000-08,5.6,953.0,323.0,0.0,6.0,NaN,0.00,97.0,0.0,NaN,75.4,84.3,66.5,33.8117,-118.1464
1,USW00023129,2000-09,5.1,1175.0,221.0,0.0,5.0,NaN,0.00,96.0,0.0,NaN,72.4,82.5,62.3,33.8117,-118.1464
2,USW00023129,2000-10,4.7,1208.0,33.0,4.0,0.0,1.0,1.81,79.0,2.3,NaN,64.7,71.9,57.5,33.8117,-118.1464
3,USW00023129,2000-11,3.6,1208.0,0.0,0.0,0.0,1.0,0.00,80.0,0.0,NaN,56.9,68.1,45.8,33.8117,-118.1464
4,USW00023129,2000-12,3.1,1209.0,0.0,0.0,0.0,NaN,0.00,80.0,0.0,NaN,57.1,68.3,46.0,33.8117,-118.1464


Use haversine to calculate the distance between two sets of coordinates to find the closest weather station to each fire

In [20]:
!pip install haversine
import haversine as hs




#each parameter is a tuple with lat, long
h = hs.haversine((33.8117, -118.1464), (33.3000,-117.35))
print(h)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
93.18494458973653


In the code below, I find the closest NOAA weather station to every fire.

In [21]:
wildfires['NOAA Station'] = ''

for index1, row in wildfires.iterrows():
  #initialize to a super large number so that condition of being less than this will be met right away
  min_dist = 100000000
  for index2, row in stations.iterrows():
    station = stations.index[index2, 'Station']
    station_coordinates = tuple(stations.at[index2, 'Latitude'], stations.at[index, 'Longitude'])
    wildfire_coordinates = tuple(wildfires.at[index1, 'Latitude'], wildfires.at[index1, 'Longitude'])
    dist = hs.haversine(station_coordinates, wildfire_coordinates)
    if dist < min_dist:
      min_dist = dist
      min_station = station
  wildfires.at[index1, 'NOAA Station'] = 

SyntaxError: ignored